# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [3]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [4]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [
    (-3 * a ** 2) - (0.5 * a ** (-0.5)),
    (3 * cos(3 * b) + 2.5 * b ** 1.5),
    (c ** (-2))
  ] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [5]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
from typing import Final


delta: Final[float] = 0.00001

numerical_grad = [
  (f(2 + delta, 3, 4) - f(2, 3, 4)) / delta,
  (f(2, 3 + delta, 4) - f(2, 3, 4)) / delta,
  (f(2, 3, 4 + delta) - f(2, 3, 4)) / delta
] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


WRONG! for dim 0: expected -12.353553390593273, yours returns -12.353612948956536
WRONG! for dim 1: expected 10.25699027111255, yours returns 10.257004202163245
OK for dim 2: expected 0.0625, yours returns 0.06249984378925432


In [6]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [
  (f(2 + delta, 3, 4) - f(2 - delta, 3, 4)) / (2 * delta),
  (f(2, 3 + delta, 4) - f(2, 3 - delta, 4)) / (2 * delta),
  (f(2, 3, 4 + delta) - f(2, 3, 4 - delta)) / (2 * delta)
] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


## section 2: support for softmax

In [7]:
# Value class starter code, with many functions taken out
from __future__ import annotations
from typing import Any
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO

  def __mul__(self, other: Value | Any) -> Value:
    other_value: Final[Value] = (other if isinstance(other, Value)
                                 else Value(other))
    result: Final[Value] = Value(self.data * other_value.data,
                                 (self, other_value), '*')

    def _backward() -> None:
      self.grad += other_value.data * result.grad
      other_value.grad += self.data * result.grad
    result._backward = _backward

    return result

  def __pow__(self, other: int | float) -> Value:
    assert isinstance(
      other, (int, float)
    ), "only supporting int/float powers for now"
    result: Final[Value] = Value(self.data ** other, (self,), f"**{other}")

    def _backward() -> None:
      self.grad += (other * self.data ** (other - 1)) * result.grad
    result._backward = _backward

    return result

  def __radd__(self, other: Value | Any) -> Value:
    return self + other

  def __neg__(self) -> Value:
    return self * (-1)

  def __truediv__(self, other: Value | Any) -> Value:
    return self * (other ** -1)

  def __rtruediv__(self, other: Value | Any) -> Value:
    return other * (self ** -1)

  def exp(self) -> Value:
    result: Final[Value] = Value(exp(self.data), (self,), "exp")

    def _backward() -> None:
      self.grad += (exp(self.data)) * result.grad
    result._backward = _backward

    return result

  def log(self) -> Value:
    result: Final[Value] = Value(log(self.data), (self,), "log")

    def _backward() -> None:
      self.grad += (1.0 / self.data) * result.grad
    result._backward = _backward

    return result

  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [8]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.nn.functional as F

logits_pt: Final[torch.Tensor] = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double()
logits_pt.requires_grad = True
probs_pt: Final[torch.Tensor] = F.softmax(logits_pt, dim=0)
loss_pt: Final[torch.Tensor] = -probs_pt[3].log()
loss_pt.backward()
print(loss_pt.data.item())

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  assert logits_pt.grad is not None
  ok = 'OK' if abs(logits_pt.grad[dim].item() - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits_pt.grad[dim].item()}")

2.1755153626167143
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625321
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400987
